# model building pipeline
## things to do
* parameterise the cfile
* generate a .cfile 
* update main.k
* call dyna from puthon on main.k
* wait for termination of dyna
* read results file.

In [179]:
import numpy as np
import os
import subprocess

from string import Template
from datetime import date, datetime

## Parameters

In [185]:
sim_name = "MM012"
lspp_template_name = 'puck_macro_template.msg.cfile'
keyword_template_name = 'mainTemplate.k'
lspp_macro_name = '{}.msg.cfile'.format(sim_name)
k_geom_name = "{}geom.k".format(sim_name)
k_main_name = "{}main.k".format(sim_name)

path_root = os.getcwd()
path_template_dir = os.path.join(path, "templates") 
path_lspp_template = os.path.join(path_template_dir, lspp_template_name)
path_k_main_template = os.path.join(path_template_dir, keyword_template_name)

path_simulation_wd = os.path.join("/media/martin/Stuff/research/MaterailModels", sim_name) 
path_lspp_macro = os.path.join(path_simulation_wd, lspp_macro_name) 
path_k_geom_file = os.path.join(path_simulation_wd, k_geom_name)
path_k_main_file = os.path.join(path_simulation_wd, k_main_name)

# macro parameters
cylinder_radius = 0.01
cylinder_length = 0.012
cylinder_elements_NumElem = int(201)

cylinder_circumference_lenght_ratio = (cylinder_radius*2*np.pi)/cylinder_length
cylinder_elements_Num = int(cylinder_elements_NumElem/cylinder_circumference_lenght_ratio)

# k file parameters
date_created = date.today()
time_created = datetime.now().strftime("%H:%M:%S")
title =  "{} [{}]".format(sim_name, date_created)
date_time = "{} ({})".format(date_created, time_created)

## check if the folder and files exits 

In [186]:
dir_list = [path_root, path_template_dir, path_simulation_wd]

for folder in dir_list:
#     print("{} {}".format(os.path.exists(folder), folder))
    try:
        os.mkdir(folder)
    except OSError:
        print("Directory Exists {}".format(folder))
    else:
        print("Directory Created {}".format(folder))

Directory Exists /home/martin/Documents/RESEARCH/musical-dollop
Directory Exists /home/martin/Documents/RESEARCH/musical-dollop/templates
Directory Exists /media/martin/Stuff/research/MaterailModels/MM012


## update cfile parameters

In [187]:
match = {
    "working_directory": path_simulation_wd,
    "keyword_file_name": path_k_geom_file}  

f = open(path_lspp_template,"r")
data = f.read() 

def replace(text, dic):
    s = Template(text)
    return s.safe_substitute(**dic)

data = replace(data, match)
# print(data)

with open(path_lspp_macro, "w") as text_file:
    print(data, file=text_file)

## Run the macro through lspp

In [188]:
bashCommand = "/home/martin/.local/bin/lspp47 c={} -nographics".format(path_lspp_macro)

process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

## Update main.K

In [190]:
match = {
    "date_time": date_time,
    "title": title,
    "include_file": path_k_file 
}  

f = open(path_k_main_template,"r")
data = f.read() 

def replace(text, dic):
    s = Template(text)
    return s.safe_substitute(**dic)

data = replace(data, match)
# print(data)

with open(path_k_main_file, "w") as text_file:
    print(data, file=text_file)

## Run lsrun 

In [223]:
# working from cmd
# /home/martin/Programs/LSTC/LSDYNA/ls-dyna_smp_s_r1010_x64_suse11_pgi165 i=/media/martin/Stuff/research/MaterailModels/MM012/MM012main.k ncpu=8 memory=1000m jobid=MM012_job02



d = dict(os.environ)
d["LSTC_LICENSE"] = 'network'
d["LSTC_LICENSE_SERVER"] = '31010@license.sun.ac.za'

bashCommand = [
    "/home/martin/Programs/LSTC/LSDYNA/ls-dyna_smp_s_r1010_x64_suse11_pgi165",
    "i=/media/martin/Stuff/research/MaterailModels/MM012/MM012main.k",
    "ncpu=8",
    "memory=1000m",
    "jobid=/media/martin/Stuff/research/MaterailModels/MM012/MM012_job01",
]

process = subprocess.Popen(bashCommand, stdout=subprocess.PIPE, env=d)
output, error = process.communicate()

In [222]:
str(31010@license.sun.ac.za)

AttributeError: '_Printer' object has no attribute 'sun'

In [224]:
output

b' License option : check network license only\n      Date: 11/13/2019      Time: 17:45:25  \n\n     ___________________________________________________\n     |                                                 |\n     |  Livermore  Software  Technology  Corporation   |\n     |                                                 |\n     |  7374 Las Positas Road                          |\n     |  Livermore, CA 94551                            |\n     |  Tel: (925) 449-2500  Fax: (925) 449-2507       |\n     |  www.lstc.com                                   |\n     |_________________________________________________|\n     |                                                 |\n     |  LS-DYNA, A Program for Nonlinear Dynamic       |\n     |  Analysis of Structures in Three Dimensions     |\n     |  Version : smp s R10.1.0   Date: 01/12/2018     |\n     |  Revision: 123264          Time: 16:34:46       |\n     |                                                 |\n     |  Features enabled in this v